In [1]:
import os
import json
import numpy as np
from matplotlib import pyplot as plt

In [2]:
uniform_horizon_dir = "../outputs/lift_lowdim_ph_reproduction/2025.04.12_11.32.56_train_diffusion_unet_lowdim_lift_lowdim_16_state_estimator/uniform_horizon/"

AHC_horizon_dir = "../outputs/lift_lowdim_ph_reproduction/2025.04.12_11.32.56_train_diffusion_unet_lowdim_lift_lowdim_16_state_estimator/AHC_total_evaluation_by_seed"

In [3]:
def get_horizon_length_success_rate(folder_dir):
    # Get all folders in uniform_horizon_dir that contain evaluation results
    folder_names = [d for d in os.listdir(folder_dir)]
    
    # folder_names = []
    
    # for d in os.listdir(folder_dir):
    #     if not "0.02" in d:
    #         folder_names.append(d)
    # folder_names.sort(key=lambda x: int(x.split('_')[3])) # Sort by horizon number

    length_and_success_rate = []

    for folder_name in folder_names:
        
        try:
            py_data = json.load(open(os.path.join(folder_dir, folder_name, "eval_log.json")))
            
            for seed in np.arange(100000, 100000+50):
                reward_key = f"test/sim_max_reward_{seed}"
                length_key = f"test/sim_action_horizon_average_length_{seed}"
                
                length_and_success_rate.append(np.array([py_data[length_key], py_data[reward_key]]))

        except Exception as e:
            print(f"Error in {os.path.join(folder_dir, folder_name, 'eval_log.json')}")
            print(e)
    length_and_success_rate = np.stack(length_and_success_rate, axis=0)
    return length_and_success_rate

In [4]:
uniform_length_and_success_rate = get_horizon_length_success_rate(uniform_horizon_dir)
AHC_length_and_success_rate = get_horizon_length_success_rate(AHC_horizon_dir)


In [5]:
unique_times = np.unique(uniform_length_and_success_rate[:, 0].round(2))
bins = unique_times-0.5
bins = np.concatenate([bins, [bins[-1]+1.0, bins[-1]+2.0]])

In [6]:
# Calculate averages and std for each bin
def get_binned_statistics(horizon_times, success, bins):
    bin_indices = np.digitize(horizon_times, bins)
    binned_success_rate = []
    bin_centers = []
    bin_counts = []
    for i in range(min(bin_indices), max(bin_indices)+1):
        mask = bin_indices == i
        if np.any(np.logical_and(mask, success)):
            bin_centers.append((bins[i-1] + bins[i]) / 2)
            binned_success_rate.append(np.mean(success[mask]))
            bin_counts.append(np.sum(mask))
        else:
            binned_success_rate.append(0)
            bin_counts.append(0)
            bin_centers.append((bins[i-1] + bins[i]) / 2)
    
    return np.array(bin_centers), np.array(binned_success_rate), bin_counts

In [7]:
uni_bin_centers, uni_binned_success_rate, uni_bin_counts = get_binned_statistics(uniform_length_and_success_rate[:, 0], uniform_length_and_success_rate[:, 1], bins)
AHC_bin_centers, AHC_binned_success_rate, AHC_bin_counts = get_binned_statistics(AHC_length_and_success_rate[:, 0]+1, AHC_length_and_success_rate[:, 1], bins)

In [ ]:
print(AHC_length_and_success_rate[:, 0]+1)

In [ ]:
print(AHC_bin_centers)

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 6))

ax1.plot(uni_bin_centers, uni_binned_success_rate, 'o:', color='tab:blue', label='Uniform')
ax1.plot(AHC_bin_centers, AHC_binned_success_rate, 'o:', color='tab:orange', label='AHC')
ax1.set_xlabel('Horizon Length')
ax1.set_ylabel('Success Rate')
ax1.set_title('Success Rate vs. Horizon Length')
ax1.legend()
ax1.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print(AHC_bin_centers)
print(AHC_bin_counts)

print(np.stack([np.array(AHC_bin_centers), np.array(AHC_bin_counts)], axis=1))

In [ ]:
import numpy as np
import os
import json

seed_number = 100041

uniform_horizon_dir = "../outputs/lift_lowdim_ph_reproduction/2025.04.12_11.32.56_train_diffusion_unet_lowdim_lift_lowdim_16_state_estimator/uniform_horizon"

AHC_horizon_dir = f"../outputs/lift_lowdim_ph_reproduction/2025.04.12_11.32.56_train_diffusion_unet_lowdim_lift_lowdim_16_state_estimator/AHC_total_evaluation_by_seed"

uniform_horizon_success = []
AHC_horizon_success = []

for horizon in range(1, 14):
    py_data = json.load(open(os.path.join(uniform_horizon_dir, f"eval_disturbance_horizon_{horizon}_distrubance_prob_1.0_vel_0.0017_gripper_dir_near_gripper_0.05_50episodes", "eval_log.json")))
    uniform_horizon_success.append(py_data[f"test/sim_max_reward_{seed_number}"])
    
    AHC_py_data = json.load(open(os.path.join(AHC_horizon_dir, f"eval_AHC_seed_{seed_number}_distrubance_prob_1.0_vel_0.0017_gripper_dir_near_gripper_0.05_50episodes", "eval_log.json")))
    AHC_horizon_success.append(AHC_py_data[f"test/sim_max_reward_{horizon}"])

uniform_horizon_success = np.array(uniform_horizon_success)
AHC_horizon_success = np.array(AHC_horizon_success)

mask = AHC_horizon_success >= 0



print(uniform_horizon_success[mask])
print(AHC_horizon_success[mask])
print(np.array(AHC_horizon_success[mask])-np.array(uniform_horizon_success[mask]))
print(np.sum(AHC_horizon_success[mask])-np.sum(uniform_horizon_success[mask]))



In [ ]:
mask

# Get Successrate by seed

In [1]:
import os
import json
import numpy as np
from matplotlib import pyplot as plt

In [2]:
seed_number = 100041

uniform_horizon_dir = "../outputs/lift_lowdim_ph_reproduction/2025.04.12_11.32.56_train_diffusion_unet_lowdim_lift_lowdim_16_state_estimator/uniform_horizon"

AHC_horizon_dir = f"../outputs/lift_lowdim_ph_reproduction/2025.04.12_11.32.56_train_diffusion_unet_lowdim_lift_lowdim_16_state_estimator/AHC_total_evaluation_by_seed"

uniform_horizon_success = []
AHC_horizon_success = []

for seed_number in range(100000, 100000+50):

    for horizon in range(1, 14):
        py_data = json.load(open(os.path.join(uniform_horizon_dir, f"eval_disturbance_horizon_{horizon}_distrubance_prob_1.0_vel_0.0017_gripper_dir_near_gripper_0.05_50episodes", "eval_log.json")))
        uniform_horizon_success.append( \
            np.array([ \
                py_data[f"test/sim_action_horizon_average_length_{seed_number}"], \
                py_data[f"test/sim_max_reward_{seed_number}"] \
            ])\
        )
        
        AHC_py_data = json.load(open(os.path.join(AHC_horizon_dir, f"eval_AHC_seed_{seed_number}_distrubance_prob_1.0_vel_0.0017_gripper_dir_near_gripper_0.05_50episodes", "eval_log.json")))
        AHC_horizon_success.append( \
            np.array([ \
                AHC_py_data[f"test/sim_action_horizon_average_length_{horizon}"], \
                AHC_py_data[f"test/sim_max_reward_{horizon}"], \
            ])\
        )

uniform_horizon_success = np.stack(uniform_horizon_success, axis=0)
AHC_horizon_success = np.stack(AHC_horizon_success, axis=0)
AHC_horizon_success = AHC_horizon_success[AHC_horizon_success[:, 0]>0]

# mask = AHC_horizon_success >= 0



# print(uniform_horizon_success[mask])
# print(AHC_horizon_success[mask])
# print(np.array(AHC_horizon_success[mask])-np.array(uniform_horizon_success[mask]))
# print(np.sum(AHC_horizon_success[mask])-np.sum(uniform_horizon_success[mask]))



In [3]:
unique_times = np.unique(uniform_horizon_success[:, 0].round(2))
bins = unique_times-0.5
bins = np.concatenate([bins, [bins[-1]+1.0, bins[-1]+2.0]])

In [ ]:
print(bins)

In [11]:
# Calculate averages and std for each bin
def get_binned_statistics(horizon_times, success, bins):
    bin_indices = np.digitize(horizon_times, bins)
    binned_success_rate = []
    bin_centers = []
    bin_counts = []
    for i in range(min(bin_indices), max(bin_indices)+1):
        mask = bin_indices == i
        if np.any(np.logical_and(mask, success)):
            bin_centers.append((bins[i-1] + bins[i]) / 2)
            binned_success_rate.append(np.mean(success[mask]))
            bin_counts.append(np.sum(mask))
        else:
            binned_success_rate.append(0)
            bin_counts.append(0)
            bin_centers.append((bins[i-1] + bins[i]) / 2)
    
    return np.array(bin_centers), np.array(binned_success_rate), bin_counts

In [ ]:
uni_bin_centers, uni_binned_success_rate, uni_bin_counts = get_binned_statistics(uniform_horizon_success[:, 0], uniform_horizon_success[:, 1], bins)
AHC_bin_centers, AHC_binned_success_rate, AHC_bin_counts = get_binned_statistics(AHC_horizon_success[:, 0], AHC_horizon_success[:, 1], bins)

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 6))

ax1.plot(uni_bin_centers, uni_binned_success_rate, 'o:', color='tab:blue', label='Uniform')
ax1.plot(AHC_bin_centers, AHC_binned_success_rate, 'o:', color='tab:orange', label='AHC')
ax1.set_xlabel('Horizon Length')
ax1.set_ylabel('Success Rate')
ax1.set_title('Success Rate vs. Horizon Length')
ax1.legend()
ax1.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
np.sum((AHC_horizon_success[:, 0] > 9.7) & (AHC_horizon_success[:, 0] < 10.3))